# 実験実行環境を構築する

このタスクでは、「新規実験」のための実行環境を構築します。  

「研究フロートップページ」に戻る場合は[こちら](../../base_FLOW.ipynb)。新規タブで開きます。  

Jupyter Notebookの操作方法については、[こちら](https://support.rdm.nii.ac.jp/usermanual/60/#jp_operation)をご覧ください。新規タブで開きます。

## 0. 研究リポジトリ名を確認する  
以下のセルを実行すると、この研究実行環境で操作する研究リポジトリ名を確認できます。  

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
utils.show_name('green')

## 1. 実験実行環境にアクセスする

以下のセルを実行した後に出力されるボタンをクリックして実験実行環境を構築してください。  
構築に成功すると自動的に「実験フロートップページ」が表示されます。  
「実験フロートップページ」の説明を参考に、「フロー図を作成する」のセルを実行し、実験フロー図を表示してください。  
実験フロー図が表示されたら、実験フローの「0. 実験フロー実行準備」フェーズの「初期セットアップを行う」をクリックし、遷移先の画面の指示にしたがって上から順に必要なセルを実行し、実験実行環境のセットアップを行ってください。  
初期セットアップまで完了する途中で、実験実行環境のページを閉じてしまった場合は、このタスクを再実行して新しい実験実行環境を構築してください。

実験実行環境について以下の点にご留意ください。  
- 実行環境が維持されるのは最後に利用した日から**30日間**です。実行環境が消えてしまった場合は、実験パッケージ配下のREADME.mdをご覧の上、新しく構築し直してください。
- 作成できる実行環境は10個までです。<br>

In [ ]:
from IPython.display import HTML
os.chdir(os.environ['HOME'] + '/WORKFLOWS')
from utils import display_util
os.chdir(os.environ['HOME'] + '/WORKFLOWS/FLOW')
from util.scripts import utils

# リモートリポジトリのURLを最新化する
is_new_private = utils.update_repo_url()

# リモートリポジトリのURLを取得する
remote_http_url = !git config --get remote.origin.url
for item in remote_http_url:
    remote_http_url = item
# 非公開リポジトリの場合ユーザー認証を行う
if is_new_private["is_private"]:
    name, password = utils.verify_GIN_user(with_email=False)
    pos = remote_http_url.find("://")
    remote_http_url = f"{remote_http_url[:pos+3]}{name}:{password}@{remote_http_url[pos+3:]}"

display_util.display_info("新規実験用の実行環境を構築します。<br>以下のボタンをクリックしてください。新規タブで開きます。<br>")
url = "https://binder.cs.rcos.nii.ac.jp/v2/git/" + urllib.parse.quote(remote_http_url, safe='') + "/HEAD?filepath=WORKFLOWS/experiment.ipynb"
HTML(f'<button onclick="window.open(\'{url}\')">実行環境を構築する</button>')

ボタンをクリックした後は、以下のような画面が表示され、実行環境の構築が開始されます。この処理には時間がかかります。<br>
構築時にエラーになってしまった場合は、[ユーザーマニュアル](https://support.rdm.nii.ac.jp/usermanual/)からデータガバナンス機能Q&A集をご覧ください。<br>

![binder_launching](../../images/binder_launching.png)

## 2. 研究フロートップページに遷移する

以下のセルを実行し、表示されるリンクをクリックして「研究フロートップページ」に戻ってください。

In [ ]:
from IPython.display import display, HTML, Javascript
display(HTML("<a href='../../base_FLOW.ipynb'>「研究フロートップページ」に遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))